In [1]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [3]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [4]:
#숫자형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [5]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [6]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [7]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [8]:
#본격적으로 시작

In [9]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")

#training = training[np.isfinite(training["GarageYrBlt"])]

In [10]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [11]:
total.head(2)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
0,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,GLQ,Unf,1.0,0.0,Gd,150.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,548.0,2.0,TA,RFn,TA,Attchd,2003.0,1710,1,GasA,Ex,2Story,1,1,Gd,Lvl,Gtl,8450,Inside,65.0,Reg,0,60,RL,196.0,BrkFace,NaN,0,2,CollgCr,61,5,7,Y,0,NaN,CompShg,Gable,Normal,208500.0,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
1,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,ALQ,Unf,0.0,1.0,Gd,284.0,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,NaN,TA,1,CBlock,2,Typ,460.0,2.0,TA,RFn,TA,Attchd,1976.0,1262,0,GasA,Ex,1Story,2,1,TA,Lvl,Gtl,9600,FR2,80.0,Reg,0,20,RL,0.0,None,NaN,0,5,Veenker,0,8,6,Y,0,NaN,CompShg,Gable,Normal,181500.0,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007


In [12]:
total.columns

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

In [13]:
#선택된 변수
category_selected = ["MSSubClass", "MSZoning", "Street" ,"LandContour", "LotConfig", 
            "Neighborhood", "BldgType", "HouseStyle", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "Foundation","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC", "CentralAir",
            "Electrical", "BsmtFullBath", "FullBath", "HalfBath", "KitchenQual",
            "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType",
            "GarageCars", "PavedDrive", "SaleCondition", "GarageFinish"]

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea', "LotArea", "MasVnrArea", "WoodDeckSF", "OpenPorchSF"]

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [14]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageType,PavedDrive,SaleCondition,GarageFinish,GarageYrBlt,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1.0,2,1,8,0,Attchd,Y,Normal,RFn,2003.0,2003,2003,706.0,150.0,856.0,856,854,1710,548.0,8450,196.0,0,61,2.0,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,NaN,Typ,208500.0
1,20,RL,Pave,Lvl,FR2,Veenker,1Fam,1Story,None,CBlock,Y,0.0,2,0,6,1,Attchd,Y,Normal,RFn,1976.0,1976,1976,978.0,284.0,1262.0,1262,0,1262,460.0,9600,0.0,298,0,2.0,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500.0


In [15]:
#임시 전처리

In [16]:
#전처리1 - 연속형데이터 중 null값이 있는 행 모두 버려주기
new_total[number_selected].isnull().sum()

GarageYrBlt     159
YearBuilt         0
YearRemodAdd      0
BsmtFinSF1        1
BsmtUnfSF         1
TotalBsmtSF       1
1stFlrSF          0
2ndFlrSF          0
GrLivArea         0
GarageArea        1
LotArea           0
MasVnrArea       23
WoodDeckSF        0
OpenPorchSF       0
dtype: int64

In [17]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())

In [18]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())

In [19]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())

In [20]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())

In [21]:
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())

In [22]:
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())

In [23]:
new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")

In [24]:
new_total = new_total.fillna("404")

In [25]:
new_total.isnull().sum()

MSSubClass       0
MSZoning         0
Street           0
LandContour      0
LotConfig        0
Neighborhood     0
BldgType         0
HouseStyle       0
MasVnrType       0
Foundation       0
CentralAir       0
BsmtFullBath     0
FullBath         0
HalfBath         0
TotRmsAbvGrd     0
Fireplaces       0
GarageType       0
PavedDrive       0
SaleCondition    0
GarageFinish     0
GarageYrBlt      0
YearBuilt        0
YearRemodAdd     0
BsmtFinSF1       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
GrLivArea        0
GarageArea       0
LotArea          0
MasVnrArea       0
WoodDeckSF       0
OpenPorchSF      0
GarageCars       0
Electrical       0
OverallQual      0
OverallCond      0
ExterQual        0
ExterCond        0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
HeatingQC        0
KitchenQual      0
FireplaceQu      0
Functional       0
SalePrice        0
dtype: int64

In [26]:
new_total.head()

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageType,PavedDrive,SaleCondition,GarageFinish,GarageYrBlt,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice
0,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1,2,1,8,0,Attchd,Y,Normal,RFn,2003.0,2003,2003,706.0,150.0,856.0,856,854,1710,548.0,8450,196,0,61,2,SBrkr,7,5,Gd,TA,Gd,TA,No,GLQ,Ex,Gd,404,Typ,208500
1,20,RL,Pave,Lvl,FR2,Veenker,1Fam,1Story,None,CBlock,Y,0,2,0,6,1,Attchd,Y,Normal,RFn,1976.0,1976,1976,978.0,284.0,1262.0,1262,0,1262,460.0,9600,0,298,0,2,SBrkr,6,8,TA,TA,Gd,TA,Gd,ALQ,Ex,TA,TA,Typ,181500
2,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1,2,1,6,1,Attchd,Y,Normal,RFn,2001.0,2001,2002,486.0,434.0,920.0,920,866,1786,608.0,11250,162,0,42,2,SBrkr,7,5,Gd,TA,Gd,TA,Mn,GLQ,Ex,Gd,TA,Typ,223500
3,70,RL,Pave,Lvl,Corner,Crawfor,1Fam,2Story,None,BrkTil,Y,1,1,0,7,1,Detchd,Y,Abnorml,Unf,1998.0,1915,1970,216.0,540.0,756.0,961,756,1717,642.0,9550,0,0,35,3,SBrkr,7,5,TA,TA,TA,Gd,No,ALQ,Gd,Gd,Gd,Typ,140000
4,60,RL,Pave,Lvl,FR2,NoRidge,1Fam,2Story,BrkFace,PConc,Y,1,2,1,9,1,Attchd,Y,Normal,RFn,2000.0,2000,2000,655.0,490.0,1145.0,1145,1053,2198,836.0,14260,350,192,84,3,SBrkr,8,5,Gd,TA,Gd,TA,Av,GLQ,Ex,Gd,TA,Typ,250000


In [27]:
#명목형 변수를 연속형으로 바꿔주는 것

In [28]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])

In [29]:
new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])

In [30]:
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [31]:
new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [32]:
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])

In [33]:
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])

In [34]:
new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [35]:
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [36]:
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])

In [37]:
new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])

In [38]:
new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
new_total.Functional.unique()

array([8, 7, 4, 6, 5, 3, 2, 0], dtype=int64)

In [39]:
#변수끼리 합쳐주는 것

In [40]:
#임시 전처리

In [41]:
new_total = new_total.assign(Is_Remod =  lambda x : x["YearBuilt"] != x["YearRemodAdd"])
new_total["Is_Remod"] = new_total.YearRemodAdd.map(lambda x : 1 if x == True else 0)
new_total["after"] = 2011 - new_total["YearRemodAdd"]
new_total = new_total.drop("YearRemodAdd", axis = 1)

In [42]:
#new_total = new_total.assign(Is_bsmt_unfinish = lambda x : x["BsmtFinType1"] == 0)
#new_total["Is_bsmt_unfinish"] = new_total.Is_bsmt_unfinish.map(lambda x : 1 if x == True else 0)

In [43]:
#new_total = new_total.assign(Is_Fireplaces = lambda x : x["Fireplaces"] != 0)
#new_total["Is_Fireplaces"] = new_total.Is_Fireplaces.map(lambda x : 1 if x == True else 0)

In [44]:
#new_total = new_total.assign(Is_GarageFinish = lambda x : x["GarageFinish"] != "Unf")
#new_total["Is_GarageFinish "] = new_total.Is_GarageFinish .map(lambda x : 1 if x == True else 0)

In [45]:
new_total["Total_Area"] = new_total["GrLivArea"] + new_total["LotArea"] + new_total["GarageArea"] + new_total["TotalBsmtSF"]

In [46]:
new_total["GrLivArea"] = new_total["GrLivArea"].map(lambda x : math.log(x))

In [47]:
new_total["LotArea"] = new_total["LotArea"].map(lambda x : math.log(x))

In [48]:
new_total["GarageArea"] = new_total["GarageArea"].map(lambda x : math.log(x+1))

In [49]:
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].map(lambda x : math.log(x+1))

In [50]:
new_total["Total_Area"] = new_total["Total_Area"].map(lambda x : math.log(x+1))

In [51]:
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].map(lambda x : math.log(x+1))

In [52]:
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].map(lambda x : math.log(x+1))

In [53]:
new_total["1stFlrSF"] = new_total["1stFlrSF"].map(lambda x : math.log(x+1))

In [54]:
new_total["2ndFlrSF"] = new_total["2ndFlrSF"].map(lambda x : math.log(x+1))

In [55]:
#new_total["is_masvnr"] = new_total.MasVnrArea.map(lambda x : 0 if int(x) == 0 else 1)
#new_total["is_wood"] = new_total.WoodDeckSF.map(lambda x : 0 if int(x) == 0 else 1)
#new_total["is_openporch"] = new_total.OpenPorchSF.map(lambda x : 0 if int(x) == 0 else 1)

In [56]:
new_total["MasVnrArea"] = new_total["MasVnrArea"].map(lambda x : math.log(int(x)+1))
new_total["WoodDeckSF"] = new_total["WoodDeckSF"].map(lambda x : math.log(int(x)+1))
new_total["OpenPorchSF"] = new_total["OpenPorchSF"].map(lambda x : math.log(int(x)+1))

In [57]:
new_total["SalePrice"] = new_total.SalePrice.map(lambda x: math.log(int(x)+1))

In [58]:
#확인

In [59]:
new_total.head(2)

,MSSubClass,MSZoning,Street,LandContour,LotConfig,Neighborhood,BldgType,HouseStyle,MasVnrType,Foundation,CentralAir,BsmtFullBath,FullBath,HalfBath,TotRmsAbvGrd,Fireplaces,GarageType,PavedDrive,SaleCondition,GarageFinish,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Total_Area
0,60,RL,Pave,Lvl,Inside,CollgCr,1Fam,2Story,BrkFace,PConc,Y,1,2,1,8,0,Attchd,Y,Normal,RFn,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,5,3,2,4,3,1,6,5,4,0,8,12.247699,0,8,9.355739
1,20,RL,Pave,Lvl,FR2,Veenker,1Fam,1Story,None,CBlock,Y,0,2,0,6,1,Attchd,Y,Normal,RFn,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,8,2,2,4,3,5,5,5,3,3,8,12.109016,0,35,9.440261


In [60]:
#히트맵으로 확인
#training = training = new_total.iloc[:len(training)]

In [61]:
#n= len(selected_with_SalePrice)
#colormap = plt.cm.viridis
#plt.figure(figsize=(n,n))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(training.corr(),
#            linewidths=0.1,
#            vmax=1.0, 
#            square=True, 
#            cmap=colormap, 
#            linecolor='white', 
#            annot=True)

In [62]:
new_total2 = dummify_category_cols(new_total,category_selected)

In [63]:
new_total2.head(2)

,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Total_Area,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_N,CentralAir_Y,BsmtFullBath_0.0,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0,FullBath_0,FullBath_1,FullBath_2,FullBath_3,FullBath_4,HalfBath_0,HalfBath_1,HalfBath_2,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13,TotRmsAbvGrd_14,TotRmsAbvGrd_15,Fireplaces_0,Fireplaces_1,Fireplaces_2,Fireplaces_3,Fireplaces_4,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf
0,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,5,3,2,4,3,1,6,5,4,0,8,12.247699,0,8,9.355739,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
1,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,8,2,2,4,3,5,5,5,3,3,8,12.109016,0,35,9.440261,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0


In [64]:
training = new_total2.iloc[:len(training)]
test = new_total2.iloc[len(training):]

In [65]:
#여기서 selected 재설정

In [66]:
training.head(2)

,GarageYrBlt,YearBuilt,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,GarageArea,LotArea,MasVnrArea,WoodDeckSF,OpenPorchSF,GarageCars,Electrical,OverallQual,OverallCond,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,KitchenQual,FireplaceQu,Functional,SalePrice,Is_Remod,after,Total_Area,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,CentralAir_N,CentralAir_Y,BsmtFullBath_0.0,BsmtFullBath_1.0,BsmtFullBath_2.0,BsmtFullBath_3.0,FullBath_0,FullBath_1,FullBath_2,FullBath_3,FullBath_4,HalfBath_0,HalfBath_1,HalfBath_2,TotRmsAbvGrd_2,TotRmsAbvGrd_3,TotRmsAbvGrd_4,TotRmsAbvGrd_5,TotRmsAbvGrd_6,TotRmsAbvGrd_7,TotRmsAbvGrd_8,TotRmsAbvGrd_9,TotRmsAbvGrd_10,TotRmsAbvGrd_11,TotRmsAbvGrd_12,TotRmsAbvGrd_13,TotRmsAbvGrd_14,TotRmsAbvGrd_15,Fireplaces_0,Fireplaces_1,Fireplaces_2,Fireplaces_3,Fireplaces_4,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,PavedDrive_N,PavedDrive_P,PavedDrive_Y,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf
0,2003.0,2003,6.561031,5.017280,6.753438,6.753438,6.751101,7.444249,6.308098,9.041922,5.283204,0.000000,4.127134,2,4,7,5,3,2,4,3,1,6,5,4,0,8,12.247699,0,8,9.355739,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0
1,1976.0,1976,6.886532,5.652489,7.141245,7.141245,0.000000,7.140453,6.133398,9.169518,0.000000,5.700444,0.000000,2,4,6,8,2,2,4,3,5,5,5,3,3,8,12.109016,0,35,9.440261,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0


In [67]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [68]:
training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [69]:
N = 5000
forest = RandomForestRegressor(n_estimators=N, n_jobs=2, max_features="sqrt", max_depth=13)

In [70]:
feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

In [71]:
forest.fit(training_set[feature], training_set[target])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=5000, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [72]:
forest.predict(test_set[feature])

array([ 12.24318407,  11.92825167,  11.94908067,  11.75683685,
        11.74541992,  12.36119534,  12.21659457,  11.93583394,
        11.80820004,  11.94166232,  12.51386993,  11.65459592,
        11.69751917,  12.2589651 ,  12.27786268,  11.72281797,
        11.76482897,  12.18104842,  12.08284169,  12.01520145,
        12.24700512,  12.109328  ,  11.43997793,  11.75796079,
        11.69024622,  12.20572711,  11.53155245,  11.83282058,
        12.67492928,  12.25218987,  11.83465095,  11.88078207,
        11.55535136,  12.25545669,  12.28449438,  11.85529658,
        12.06969225,  11.92299475,  12.40399916,  12.27599439,
        11.83752931,  11.59003158,  11.8241588 ,  12.52515233,
        12.38781217,  11.69765154,  11.90814602,  11.9875885 ,
        11.83787332,  11.69345996,  11.85253027,  12.34543014,
        12.11113442,  12.08076867,  12.49930976,  11.56996246,
        11.4825329 ,  12.30114401,  12.20686067,  12.68053076,
        11.71537955,  12.13068175,  11.66321119,  12.45

In [73]:
forest.score(test_set[feature], test_set[target])

0.87897974255678801

In [74]:
y_pred=forest.predict(test_set[feature])

In [75]:
y_pred = np.array(list(map(lambda x: math.exp(x)-1,y_pred)))

In [76]:
y_true = test_set["SalePrice"]
y_ture= np.array(list(map(lambda x: math.exp(x)-1,y_true)))

In [77]:
y_true = test_set["SalePrice"]
rmse = RMSE(np.exp(y_true),y_pred)
print(rmse)

24924.27437362485


In [78]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)

In [79]:
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

In [72]:
#최적화

In [ ]:
# 최적화할 파라미터 N
best_score = 0
for N in range(4000,8500,500):
    tmp = RandomForestRegressor(n_estimators=N)
    tmp.fit(training_set[feature], training_set[target])
    score = tmp.score(test_set[feature],test_set[target])
    print("최적화 n_estimator 값 : {}".format(N))
    print("예측률 : {}".format(score))
    
    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
    if score>best_score:
        best_score = score
        best_N = N
        best_rf_model = tmp
        
print("최적화 n_estimator 값 : {}".format(best_N))
print("예측률 : {}".format(best_score))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 4000
예측률 : 0.9042194822640276


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 4500
예측률 : 0.9035964279289123


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 5000
예측률 : 0.9043838468558979


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



최적화 n_estimator 값 : 5500
예측률 : 0.9042545445216248


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



In [74]:
#best_score = 0
#for N in range(13,21):
#    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
#    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 max_depth 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [75]:
#제출

In [80]:
forest.fit(training[feature], training[target])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=13,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=5000, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [81]:
prediction =forest.predict(test[feature])

In [82]:
prediction = np.array(list(map(lambda x: math.exp(x)-1,prediction)))

In [83]:
submission = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction})

In [84]:
submission.to_csv("Submission41.csv")